In [1]:
import os

os.environ['VISIBLE_CUDA_DEVICES'] = '1'

In [2]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
)


In [3]:
# load model mistralai/Mistral-7B-v0.1 and tokenizer
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1", device='cuda:0')
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")



Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
model

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
    (norm): MistralRM

In [6]:
from prettytable import PrettyTable

def model_size_and_parameters(model):
    # Create a PrettyTable for displaying module-wise parameter information
    table = PrettyTable(["Modules", "Parameters"])

    # Calculate the total number of parameters in the model
    model_size = sum(t.numel() for t in model.parameters())

    # Print the total size of the model in megabytes
    print(f"bert-base-uncased size: {model_size/1000**2:.1f}M parameters")

    # Initialize a variable to keep track of the total trainable parameters
    total_params = 0

    # Iterate through named parameters of the model
    for name, parameter in model.named_parameters():
        # Check if the parameter requires gradient (i.e., is trainable)
        if not parameter.requires_grad:
            continue

        # Get the number of parameters in the current module
        params = parameter.numel()

        # Add a row to the PrettyTable with module name and number of parameters
        table.add_row([name, params])

        # Increment the total trainable parameters
        total_params += params

    # Print the PrettyTable with module-wise parameter information
    print(table)

    # Print the total number of trainable parameters in the model
    print(f"Total Trainable Params: {total_params}")

    # Return the total number of trainable parameters
    return total_params

model_size_and_parameters(model)


bert-base-uncased size: 7241.7M parameters
+-------------------------------------------------+------------+
|                     Modules                     | Parameters |
+-------------------------------------------------+------------+
|            model.embed_tokens.weight            | 131072000  |
|      model.layers.0.self_attn.q_proj.weight     |  16777216  |
|      model.layers.0.self_attn.k_proj.weight     |  4194304   |
|      model.layers.0.self_attn.v_proj.weight     |  4194304   |
|      model.layers.0.self_attn.o_proj.weight     |  16777216  |
|       model.layers.0.mlp.gate_proj.weight       |  58720256  |
|        model.layers.0.mlp.up_proj.weight        |  58720256  |
|       model.layers.0.mlp.down_proj.weight       |  58720256  |
|      model.layers.0.input_layernorm.weight      |    4096    |
|  model.layers.0.post_attention_layernorm.weight |    4096    |
|      model.layers.1.self_attn.q_proj.weight     |  16777216  |
|      model.layers.1.self_attn.k_proj.weight  

7241732096

In [4]:
model.to('cuda:1')

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
    (norm): MistralRM

# Loading mistral model with custom config from huggingface

In [16]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
config = AutoConfig.from_pretrained("mistralai/Mistral-7B-v0.1")
from transformers import MistralConfig
ModelParam_M,D_emb,Vocal,D_Head,d_FF,N_Layer,N_Head,KV_Head,Window,GPU_use_MB = 1607.8,4096,50000,128,14336,4,16,4,4096,6532.25
custom_config = MistralConfig(
        vocab_size=Vocal,
        hidden_size=D_emb,
        intermediate_size=d_FF,
        num_hidden_layers=N_Layer,
        num_attention_heads=N_Head,
        num_key_value_heads=KV_Head,
        hidden_act="silu",
        max_position_embeddings=4096 * 32,
        initializer_range=0.02,
        rms_norm_eps=1e-6,
        use_cache=True,
        pad_token_id=None,
        bos_token_id=1,
        eos_token_id=2,
        tie_word_embeddings=False,
        rope_theta=10000.0,
        sliding_window=Window,
        attention_dropout=0.0,
    )

model_custom = AutoModelForCausalLM.from_config(custom_config)
model_custom.num_parameters()


1282052096

In [1]:
from transformers import MistralForCausalLM, MistralConfig


In [3]:
D_emb,Vocal,d_head,d_FF,N_Layer,N_Head,KV_Head,Window = 4096,50000,128,14336,2,32,8,8192

custom_config = MistralConfig(
        vocab_size=Vocal,
        hidden_size=D_emb,
        intermediate_size=d_FF,
        num_hidden_layers=N_Layer,
        num_attention_heads=N_Head,
        num_key_value_heads=KV_Head,
        hidden_act="silu",
        max_position_embeddings=4096 * 32,
        initializer_range=0.02,
        rms_norm_eps=1e-6,
        use_cache=True,
        pad_token_id=None,
        bos_token_id=1,
        eos_token_id=2,
        tie_word_embeddings=False,
        rope_theta=10000.0,
        sliding_window=Window,
        attention_dropout=0.0,
    )

In [4]:
model = MistralForCausalLM(custom_config)

In [12]:
model.num_parameters()/1000**2

845.828096